dataset link : [https://drive.google.com/file/d/1X4Hcj72NK7J2JYvgjICFj0R1XwUq1w0a/view](https://)

In [1]:
path = "/content/100_Unique_QA_Dataset.csv"

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(path)

In [4]:
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [5]:
def tokenize(row):
  row = row.lower()
  row = row.replace("?"," ")
  row = row.replace("'"," ")
  return row.split()

In [6]:
tokenize("what is the capital of france?")

['what', 'is', 'the', 'capital', 'of', 'france']

In [8]:
# vocabulary
vocab = {"<unk>":0}

In [9]:
def build_vocabulary(row):
  tokenized_question = tokenize(row["question"])
  tokenized_answer = tokenize(row["answer"])
  sentence = tokenized_question + tokenized_answer
  for token in sentence:
    if token not in vocab:
      vocab[token] = len(vocab)

In [10]:
df.apply(build_vocabulary,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [11]:
len(vocab)

324

In [12]:
vocab

{'<unk>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [19]:
def tokenize_index(row,vocab):
  index_list = []
  for token in tokenize(row):
    if token in vocab:
      index_list.append(vocab[token])
    else:
      index_list.append(vocab['<unk>'])
  return index_list

In [20]:
tokenize_index("what is capital of france?",vocab)

[1, 2, 4, 5, 6]

In [18]:
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn

In [21]:
class QADataset(Dataset):
  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab
  def __len__(self):
    return self.df.shape[0]
  def __getitem__(self,index):
    question = tokenize_index(self.df.iloc[index]["question"],vocab)
    answer = tokenize_index(self.df.iloc[index]["answer"],vocab)
    return torch.tensor(question),torch.tensor(answer)

In [22]:
dataset = QADataset(df,vocab)

In [23]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [24]:
dataset[10]

(tensor([ 1,  2,  3,  4,  5, 53]), tensor([54]))

In [25]:
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [73]:
class model(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn = nn.RNN(50,hidden_size=64,batch_first=True)
    self.linear = nn.Linear(64,vocab_size)
  def forward(self,x):
    x = self.embedding(x)
    hidden,final = self.rnn(x)
    return self.linear(final.squeeze(0))

In [74]:
rnn_model = model(len(vocab))

In [ ]:
rnn_model(dataloader[0])

In [37]:
for x,y in dataloader:
  print(x.shape,y.shape)
  break

torch.Size([1, 9]) torch.Size([1, 1])


In [65]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64,batch_first=True) # to get that the final(d) as only the last occurance
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [47]:
c.shape,d.shape

(torch.Size([1, 6, 64]), torch.Size([1, 1, 64]))

In [62]:
c,d

(tensor([[[ 6.3264e-01, -3.5769e-01, -4.2828e-01,  1.5967e-01,  1.1467e-01,
            4.2389e-01,  2.6980e-01, -1.9390e-01,  2.6303e-01,  2.6533e-01,
           -2.9741e-01, -4.7098e-01,  1.6072e-01, -6.0927e-01, -5.2474e-04,
            3.3287e-01,  4.9862e-01, -3.0195e-01, -1.7806e-01, -2.1905e-01,
            4.8770e-01, -3.6224e-01, -1.1149e-01,  3.4531e-01,  7.6989e-01,
            3.9448e-01,  2.9424e-01,  6.5128e-01,  4.3278e-01,  4.3293e-01,
           -7.4180e-02, -4.3649e-01, -2.1862e-02, -3.6504e-01, -4.4899e-02,
           -6.4405e-01,  1.8321e-01,  2.7709e-02, -6.7488e-01, -7.7164e-02,
           -7.5402e-01, -3.5074e-02, -2.1957e-01,  3.9544e-01, -1.6429e-02,
           -5.0874e-01, -5.0584e-01,  2.6234e-01, -5.7353e-01, -4.2865e-01,
           -5.9423e-02, -6.1964e-02,  4.5222e-01, -2.6996e-02,  5.3350e-01,
           -1.9045e-01,  4.1853e-01,  7.6755e-01, -1.7429e-01,  3.9937e-01,
           -3.8475e-01, -3.9095e-01,  1.7220e-02,  7.1989e-01],
          [ 1.2430e-01, 

In [64]:
e.squeeze(0)

tensor([[ 7.0501e-02, -1.4555e-01, -1.6593e-01,  8.7636e-02, -1.5836e-01,
         -3.2300e-02, -5.0766e-01,  9.6510e-02,  5.0797e-01, -5.8258e-01,
          1.8804e-01,  1.2873e-01, -6.3309e-02, -7.5379e-02, -3.4963e-01,
          3.8947e-01, -2.2313e-01,  3.9108e-02, -8.3202e-02,  2.0071e-01,
          2.9850e-01, -4.8496e-02,  3.0473e-01, -2.8086e-01,  5.7007e-02,
          5.1580e-01,  4.7157e-01,  2.8018e-01, -3.8047e-01,  1.6524e-01,
         -1.6551e-01, -1.2602e-01,  9.6752e-02,  1.5783e-01,  2.7570e-01,
         -2.1586e-01, -3.2147e-02, -1.1390e-01, -2.7417e-01,  2.5986e-02,
          1.2590e-01, -3.1699e-01, -1.4163e-01, -3.0561e-01, -1.1990e-01,
          1.0782e-01, -1.4830e-01, -4.3994e-04,  1.0184e-01, -5.2345e-02,
          1.3092e-01, -7.1893e-02,  1.6092e-02,  2.8581e-01,  1.6281e-01,
         -8.6215e-02,  2.7909e-01,  2.7870e-01,  5.0636e-02,  1.1029e-01,
         -3.2752e-02, -2.9885e-01, -4.9913e-01, -2.6338e-01, -2.9730e-03,
         -7.1681e-01, -2.0596e-01,  1.

In [75]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn_model.parameters(),lr = 0.001)

In [78]:
## training model
epochs = 50
for epoch in range(epochs):
  total_loss = 0
  for question,answer in dataloader:
    outputs = rnn_model(question)
    loss = criterion(outputs,answer[0])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss += loss.item()
  print(f"Epoch:{epoch+1},loss:{total_loss:.4f}")

Epoch:1,loss:63.4034
Epoch:2,loss:50.2389
Epoch:3,loss:40.3892
Epoch:4,loss:32.7829
Epoch:5,loss:27.0603
Epoch:6,loss:22.8494
Epoch:7,loss:19.3895
Epoch:8,loss:16.4901
Epoch:9,loss:14.2019
Epoch:10,loss:12.3678
Epoch:11,loss:10.8132
Epoch:12,loss:9.4169
Epoch:13,loss:8.4252
Epoch:14,loss:7.4405
Epoch:15,loss:6.6519
Epoch:16,loss:5.9610
Epoch:17,loss:5.3313
Epoch:18,loss:4.8502
Epoch:19,loss:4.4120
Epoch:20,loss:4.0360
Epoch:21,loss:3.7022
Epoch:22,loss:3.3868
Epoch:23,loss:3.1164
Epoch:24,loss:2.8763
Epoch:25,loss:2.6693
Epoch:26,loss:2.4729
Epoch:27,loss:2.2932
Epoch:28,loss:2.1400
Epoch:29,loss:1.9992
Epoch:30,loss:1.8608
Epoch:31,loss:1.7403
Epoch:32,loss:1.6281
Epoch:33,loss:1.5305
Epoch:34,loss:1.4341
Epoch:35,loss:1.3462
Epoch:36,loss:1.2653
Epoch:37,loss:1.1883
Epoch:38,loss:1.1213
Epoch:39,loss:1.0575
Epoch:40,loss:0.9976
Epoch:41,loss:0.9417
Epoch:42,loss:0.8897
Epoch:43,loss:0.8407
Epoch:44,loss:0.7945
Epoch:45,loss:0.7517
Epoch:46,loss:0.7114
Epoch:47,loss:0.6731
Epoch:48,lo

In [83]:
rnn_model(torch.tensor(tokenize_index("what is the captial of france",vocab))).unsq

torch.Size([324])

In [89]:
def predict_answer(model,question,threshold=0.5):
  tokenized_question = tokenize_index(question,vocab)
  question_tensor = torch.tensor(tokenized_question).unsqueeze(0)
  output = rnn_model(question_tensor)
  probs = torch.nn.functional.softmax(output)
  value,index = torch.max(probs,dim=1)
  if value<threshold:
    print(f"I don't know")
  else:
    print(list(vocab.keys())[index])


In [90]:
predict_answer(rnn_model, "What is the largest planet in our solar system?")

jupiter


/tmp/ipython-input-3671193018.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = torch.nn.functional.softmax(output)


In [91]:
predict_answer(rnn_model, "What is the capital of Spain?")

madrid


/tmp/ipython-input-3671193018.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = torch.nn.functional.softmax(output)
